<a href="https://colab.research.google.com/github/khan-habibullah/ml1/blob/master/feature_extraction_resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
import cv2
from skimage.feature import local_binary_pattern
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import os

shoes_dir = '/content/Shoe vs Sandal vs Boot Dataset/Shoe'
sandals_dir = '/content/Shoe vs Sandal vs Boot Dataset/Sandal'
boots_dir = '/content/Shoe vs Sandal vs Boot Dataset/Boot'

def extract_resnet_features(img_path):
# Load image and preprocess for ResNet50
  img = load_img(img_path, target_size=(224, 224))
  x = img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = preprocess_input(x)
# Extract features using ResNet50
  model = ResNet50(weights='imagenet', include_top=False)
  features = model.predict(x)
  return features.flatten()
def extract_lbp_features(img_path):
# Load image and convert to grayscale
  img = cv2.imread(img_path)
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# Extract LBP features
  lbp = local_binary_pattern(gray, 8, 1)
  hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, 257), range=(0, 256))
  hist = hist.astype("float")
  hist /= (hist.sum() + 1e-7)
  return hist
def extract_image_dimensions(img_path):
# Load image and get dimensions
  img = cv2.imread(img_path)
  height, width, _ = img.shape
  return [height, width]
resnet_features = []
lbp_features = []
image_dims = []
labels = []
for folder, label in [(shoes_dir, 0), (sandals_dir, 1), (boots_dir, 2)]:
  for filename in os.listdir(folder):
    if filename.endswith('.jpg'):
# Extract features and dimensions for image
      img_path = os.path.join(folder, filename)
      resnet_feature = extract_resnet_features(img_path)
      lbp_feature = extract_lbp_features(img_path)
      image_dim = extract_image_dimensions(img_path)
## Append features and dimensions to lists
      resnet_features.append(resnet_feature)
      lbp_features.append(lbp_feature)
      image_dims.append(image_dim)
      labels.append(label)

#Convert lists to arrays
resnet_features = np.array(resnet_features)
lbp_features = np.array(lbp_features)
image_dims = np.array(image_dims)
labels = np.array(labels)

#Create dataframes from arrays
resnet_df = pd.DataFrame(resnet_features)
lbp_df = pd.DataFrame(lbp_features)
dims_df = pd.DataFrame(image_dims)
labels_df = pd.DataFrame(labels, columns=['label'])

#Concatenate dataframes
features_df = pd.concat([resnet_df, lbp_df, dims_df], axis=1)

#Save dataframes to CSV files
features_df.to_csv('features.csv', index=False)
labels_df.to_csv('labels.csv', index=False)

1/1 [==============================] - 1s 1s/step


1/1 [==============================] - 1s 1s/step


1/1 [==============================] - 2s 2s/step


KeyboardInterrupt: ignored

In [3]:
import zipfile
with zipfile.ZipFile("/content/archive (5).zip","r") as zip_ref:
    zip_ref.extractall("/content/")